In [30]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
df = pd.read_csv("./data/gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df =df.drop(labels=['id'], axis=1)

In [4]:
X = df.drop(labels=['price'], axis = 1)

In [5]:
y = df [['price']]
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [6]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   carat    193573 non-null  float64
 1   cut      193573 non-null  object 
 2   color    193573 non-null  object 
 3   clarity  193573 non-null  object 
 4   depth    193573 non-null  float64
 5   table    193573 non-null  float64
 6   x        193573 non-null  float64
 7   y        193573 non-null  float64
 8   z        193573 non-null  float64
 9   price    193573 non-null  int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 14.8+ MB


In [8]:
color_categories = ['F', 'J', 'G', 'E', 'D', 'H', 'I']
cut_categories = ['Premium', 'Very Good', 'Ideal', 'Good', 'Fair']
clarity_categories = ['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1']


In [9]:
numerical_columns = X.select_dtypes(exclude='object').columns
categorical_columns = X.select_dtypes(include='object').columns
print(numerical_columns, categorical_columns)

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object') Index(['cut', 'color', 'clarity'], dtype='object')


In [10]:
num_pipeline = Pipeline(
  steps = [
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
  ]
)

cat_pipeline = Pipeline(
steps=[
  ('imputer', SimpleImputer(strategy="most_frequent")),
  ('encoding', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
]
)

In [11]:
proprocessor = ColumnTransformer(
  [
    ('num_pipeline', num_pipeline, numerical_columns),
    ('cat_pipeline', cat_pipeline, categorical_columns)
  ]
)



In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=30)

In [24]:
X_train.head()

,carat,cut,color,clarity,depth,table,x,y,z
168192,0.34,Ideal,I,VVS2,60.9,57.0,4.56,4.53,2.76
35202,0.90,Good,E,SI1,63.8,57.0,6.07,6.03,3.87
41091,1.02,Premium,G,VS1,62.7,58.0,6.35,6.39,4.00
31239,0.32,Premium,G,VS2,62.1,59.0,4.37,4.35,2.71
45722,0.35,Ideal,J,VVS2,61.1,56.0,4.53,4.57,2.78


In [25]:
X_train = pd.DataFrame(proprocessor.fit_transform(X_train), columns= proprocessor.get_feature_names_out() )
X_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,2.0,6.0,5.0
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,3.0,3.0,3.0
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,0.0,2.0,2.0
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,0.0,2.0,0.0
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,2.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...
135496,-1.040295,-0.016876,-0.642862,-1.268122,-1.244270,-1.239078,2.0,3.0,3.0
135497,0.991842,0.168176,-0.642862,1.048629,1.114501,1.079486,2.0,2.0,1.0
135498,0.451380,1.556060,-0.642862,0.516768,0.588314,0.702719,3.0,2.0,3.0
135499,0.667565,-1.774863,1.442462,0.868337,0.951202,0.688228,2.0,2.0,2.0


In [26]:
X_test = pd.DataFrame(proprocessor.transform(X_test), columns= proprocessor.get_feature_names_out() )
X_test

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.564688,-0.942132,-0.642862,-0.429765,-0.464061,-0.500036,0.0,3.0,0.0
1,-0.175556,1.000906,-0.121531,-0.042137,-0.028595,0.036132,1.0,5.0,3.0
2,-1.061913,0.260701,-0.121531,-1.304180,-1.298703,-1.268060,2.0,5.0,4.0
3,0.970223,-0.201927,1.963794,1.048629,0.996563,0.978049,0.0,2.0,0.0
4,-0.932202,-1.312235,0.399800,-1.006699,-0.990248,-1.065186,0.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...
58067,1.013460,1.185958,-0.642862,1.003556,1.041924,1.151941,1.0,5.0,0.0
58068,-0.997058,0.260701,-1.164193,-1.141917,-1.126331,-1.108659,2.0,0.0,6.0
58069,-0.197174,-3.347799,1.442462,0.102096,0.071199,-0.224706,0.0,1.0,0.0
58070,-0.824110,-0.201927,-0.121531,-0.853450,-0.881382,-0.876803,2.0,2.0,3.0


In [36]:
# y_train = np.array(y_train)
# y_test = np.array(y_test)
# print(y_train.shape, y_test.shape)

(135501, 1) (58072, 1)


In [41]:
models = {
  'LinearRegression' : LinearRegression(),
  'Ridge' : Ridge(),
  'Lasso' : Lasso(),
  'ElasticNet' : ElasticNet(),
  'SVR' : SVR(),
  # 'RandomForestRegressor' : RandomForestRegressor()

}

model_list = []
trained_model_list = []
r2_list =[]


for i in models.keys():
  model = models[i]
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)

  mae = mean_absolute_error(y_test, y_pred)
  rmse = mean_squared_error(y_test, y_pred, squared=False)
  r2 = r2_score(y_test, y_pred)

  model_list.append(i)
  trained_model_list.append(model)
  r2_list.append(r2)

  print(i)
  print("Mean absolute error : ", mae)
  print("Mean squared error : ", rmse)
  print("r2 score : ", r2)

  print()
  print("--"*20)
  print()

LinearRegression
Mean absolute error :  726.0528982162504
Mean squared error :  1198.8293111469354
r2 score :  0.9117706806392406

----------------------------------------

Ridge
Mean absolute error :  726.0819590791149
Mean squared error :  1198.8305077010427
r2 score :  0.9117705045154069

----------------------------------------

Lasso
Mean absolute error :  727.5667075725914
Mean squared error :  1198.713796345963
r2 score :  0.9117876827281106

----------------------------------------

ElasticNet
Mean absolute error :  1133.4879105147706
Mean squared error :  1629.5349717231074
r2 score :  0.8369857410695969

----------------------------------------



d:\data_science\ml_projects\ml_bootcamp_project\myenv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [38]:
trained_model_list

[LinearRegression(), Ridge(), Lasso(), ElasticNet()]

In [39]:
model_list

['LinearRegression', 'Ridge', 'Lasso', 'ElasticNet']

In [40]:
r2_list

[0.9117706806392406,
 0.9117705045154069,
 0.9117876827281106,
 0.8369857410695969]